In [4]:
!pip install filterpy
  # Install scikit-learn to install datasets as dependencies
!pip install pynwb
!pip install hmmlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=95ebb5ddf752af330d8c479b7ac54e46ea18433848e569db4a7e7df72aac6e29
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.5/338.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 3.1 MB/s eta 0:00:00


In [9]:
import json
import numpy as np
import pandas as pd
with open("/content/drive/MyDrive/MLNBD/ML_Neuro/DATA ANALYTICS/trajectory_with_spike_data.json", "r") as f:
    data = json.load(f)

In [5]:
from pynwb import NWBHDF5IO

# Load NWB file
io = NWBHDF5IO('/content/drive/MyDrive/MLNBD/ML_Neuro/Original_Data/sub-Jenkins_ses-small_desc-train_behavior+ecephys.nwb', 'r')
nwbfile = io.read()

/usr/local/lib/python3.10/dist-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/usr/local/lib/python3.10/dist-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/usr/local/lib/python3.10/dist-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [10]:

# Access behavior data
hand_pos = nwbfile.processing['behavior'].data_interfaces['hand_pos']
hand_vel = nwbfile.processing['behavior'].data_interfaces['hand_vel']
eye_pos = nwbfile.processing['behavior'].data_interfaces['eye_pos']
cursor_pos = nwbfile.processing['behavior'].data_interfaces['cursor_pos']
trials = nwbfile.intervals['trials']

In [13]:
units = nwbfile.units  # Access the units table
spike_times = units['spike_times']  # Spike times for each unit
obs_intervals = units['obs_intervals']  # Observation intervals for each unit

In [12]:
trial_data = []
for trial_idx in range(len(trials)):
    start_time = trials["start_time"][trial_idx]
    stop_time = trials["stop_time"][trial_idx]
    trial_type = trials["trial_type"][trial_idx]
    target_pos = trials["target_pos"][trial_idx]
    barrier_pos = trials["barrier_pos"][trial_idx]

    time_mask = np.logical_and(
        hand_pos["timestamps"] >= start_time,
        hand_pos["timestamps"] <= stop_time,
    )
    trial_hand_pos = hand_pos["data"][time_mask]
    trial_hand_timestamps = hand_pos["timestamps"][time_mask]
    trial_hand_vel = hand_vel["data"][time_mask]
    trial_eye_pos = eye_pos["data"][time_mask]
    trial_cursor_pos = cursor_pos["data"][time_mask]

    trial_data.append({
        "trial_id": trial_idx,
        "start_time": start_time,
        "stop_time": stop_time,
        "trial_type": trial_type,
        "target_pos": target_pos,
        "barrier_pos": barrier_pos,
        "hand_pos": trial_hand_pos.tolist(),
        "hand_timestamps": trial_hand_timestamps.tolist(),
        "hand_vel": trial_hand_vel.tolist(),
        "eye_pos": trial_eye_pos.tolist(),
        "cursor_pos": trial_cursor_pos.tolist(),
        "spikes": {k: v["data"][time_mask].tolist() for k, v in spikes.items()},
    })

# Convert to a structured DataFrame
trial_spike_data = []

for trial in trial_data:
    for i, timestamp in enumerate(trial["hand_timestamps"]):
        row = {
            "trial_id": trial["trial_id"],
            "timestamp": timestamp,
            "hand_pos": trial["hand_pos"][i],
            "hand_vel": trial["hand_vel"][i],
            "eye_pos": trial["eye_pos"][i],
            "cursor_pos": trial["cursor_pos"][i],
            "target_pos": trial["target_pos"],
            "barrier_pos": trial["barrier_pos"],
            "trial_type": trial["trial_type"],
        }
    start_time = trials['start_time'][trial_idx]
    stop_time = trials['stop_time'][trial_idx]

    # Filter spike times for this trial
    trial_spikes = []
    for unit_idx in range(len(units)):
        # Get spike times for this unit
        unit_spike_times = spike_times[unit_idx]

        # Filter spike times within the trial interval
        trial_unit_spikes = unit_spike_times[
            (unit_spike_times >= start_time) & (unit_spike_times <= stop_time)
        ]

        # Append the unit-specific spikes for this trial
        trial_spikes.append(trial_unit_spikes.tolist())

    # Store trial-level spike data
    trial_spike_data.append({
        "trial_id": trial_idx,
        "start_time": start_time,
        "stop_time": stop_time,
        "spike_data": trial_spikes  # List of spike times for all units
    })
structured_data = pd.DataFrame(rows)


TypeError: 'SpatialSeries' object is not subscriptable

In [14]:
import numpy as np

trial_data = []
for trial_idx in range(len(trials)):
    # Extract trial-specific information
    trial_type = int(trials['trial_type'][trial_idx])
    start_time = trials['start_time'][trial_idx]
    stop_time = trials['stop_time'][trial_idx]
    target_pos = trials['target_pos'][trial_idx]
    barrier_pos = trials['barrier_pos'][trial_idx]

    # Filter behavioral data for this trial interval
    time_mask = np.logical_and(
        hand_pos.timestamps >= start_time,
        hand_pos.timestamps <= stop_time
    )
    trial_hand_pos = hand_pos.data[time_mask]
    trial_hand_vel = hand_vel.data[time_mask]
    trial_eye_pos = eye_pos.data[time_mask]
    trial_cursor_pos = cursor_pos.data[time_mask]

    # Handle missing data
    trial_hand_pos = trial_hand_pos.tolist() if trial_hand_pos.size > 0 else []
    trial_hand_vel = trial_hand_vel.tolist() if trial_hand_vel.size > 0 else []
    trial_eye_pos = trial_eye_pos.tolist() if trial_eye_pos.size > 0 else []
    trial_cursor_pos = trial_cursor_pos.tolist() if trial_cursor_pos.size > 0 else []

    # Filter spike data for this trial interval
    trial_spikes = []
    for unit_idx in range(len(units)):
        unit_spike_times = units['spike_times'][unit_idx]

        # Filter spike times for the current trial
        trial_unit_spikes = unit_spike_times[
            (unit_spike_times >= start_time) & (unit_spike_times <= stop_time)
        ]

        # Append spike data for this unit
        trial_spikes.append(trial_unit_spikes.tolist())

    # Store trial data
    trial_data.append({
        "trial_id": trial_idx,
        "start_time": start_time,
        "stop_time": stop_time,
        "trial_type": trial_type,
        "target_pos": np.array(target_pos).tolist(),
        "barrier_pos": np.array(barrier_pos).tolist(),
        "hand_pos": trial_hand_pos,
        "hand_vel": trial_hand_vel,
        "eye_pos": trial_eye_pos,
        "cursor_pos": trial_cursor_pos,
        "spike_data": trial_spikes  # List of spike times for all units
    })

# Inspect trial data (optional)
print(f"Processed {len(trial_data)} trials.")
print("Example Trial Data:", trial_data[0])


Processed 100 trials.
Example Trial Data: {'trial_id': 0, 'start_time': 0.0, 'stop_time': 3.421, 'trial_type': 11, 'target_pos': [[-118, -83]], 'barrier_pos': [[-33, 47, 37, 6], [-77, 48, 61, 11], [-64, -22, 8, 66], [-188, -115, 5, 40], [0, 104, 11, 46], [86, -40, 20, 11], [85, -96, 6, 59], [149, -18, 51, 6], [-76, -29, 11, 40]], 'hand_pos': [[-4.069391463135143, -31.888892887357343], [-4.066940743020508, -31.90072273920979], [-4.064521096636136, -31.912980443054277], [-4.062168204665466, -31.92550113590004], [-4.059822094964986, -31.93811222659131], [-4.0575018514898025, -31.950827609315958], [-4.055336235211395, -31.963551494181534], [-4.053194103875741, -31.976226125096254], [-4.051181679682036, -31.98881230124836], [-4.049442752983631, -32.001309261157814], [-4.047961747944242, -32.01360252720889], [-4.046694205371443, -32.025667403302066], [-4.045650269799997, -32.037435315863426], [-4.044949639691285, -32.048996963288324], [-4.044444994545661, -32.06038109964912], [-4.04422184744

In [16]:
rows = []

# Extract spike data separately
spike_times = units['spike_times']  # Assuming this is a list of arrays
spike_units = len(spike_times)

for trial in trial_data:
    trial_id = trial["trial_id"]
    trial_type = trial["trial_type"]
    trial_timestamps = hand_pos.timestamps  # Use `.timestamps` attribute
    trial_hand_pos = trial["hand_pos"]

    for i, timestamp in enumerate(trial_timestamps):
        if i < len(trial_hand_pos):  # Match indices
            row = {
                "trial_id": trial_id,
                "trial_type": trial_type,
                "timestamp": timestamp,
                "hand_pos": trial_hand_pos[i],
            }
            # Add spike data for each unit
            for unit_idx in range(spike_units):
                unit_spike_times = spike_times[unit_idx]
                # Count spikes that occurred at this timestamp
                spikes_at_time = len([t for t in unit_spike_times if t == timestamp])
                row[f"spike_{unit_idx + 1}"] = spikes_at_time
            rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

# Display to user
import ace_tools as tools; tools.display_dataframe_to_user(name="Trial Time-HandPos-Spike DataFrame", dataframe=df)


KeyboardInterrupt: 

In [17]:
rows = []

# Extract spike data separately
spike_times = units['spike_times']  # Assuming this is a list of arrays
spike_units = len(spike_times)

for trial in trial_data:
    trial_id = trial["trial_id"]
    trial_type = trial["trial_type"]
    trial_timestamps = hand_pos.timestamps  # Use `.timestamps` attribute
    trial_hand_pos = trial["hand_pos"]
    trial_hand_vel = trial["hand_vel"]
    trial_eye_pos = trial["eye_pos"]
    trial_cursor_pos = trial["cursor_pos"]
    target_pos = trial["target_pos"]
    barrier_pos = trial["barrier_pos"]

    for i, timestamp in enumerate(trial_timestamps):
        if i < len(trial_hand_pos):  # Match indices
            row = {
                "trial_id": trial_id,
                "timestamp": timestamp,
                "hand_pos": trial_hand_pos[i],
                "hand_vel": trial_hand_vel[i] if i < len(trial_hand_vel) else None,
                "eye_pos": trial_eye_pos[i] if i < len(trial_eye_pos) else None,
                "cursor_pos": trial_cursor_pos[i] if i < len(trial_cursor_pos) else None,
                "target_pos": target_pos,
                "barrier_pos": barrier_pos,
                "trial_type": trial_type,
            }
            # Add spike data for each unit
            for unit_idx in range(spike_units):
                unit_spike_times = spike_times[unit_idx]
                # Count spikes that occurred at this timestamp
                spikes_at_time = len([t for t in unit_spike_times if t == timestamp])
                row[f"spike_{unit_idx + 1}"] = spikes_at_time
            rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

# Display to user
import ace_tools as tools; tools.display_dataframe_to_user(name="Comprehensive Trial DataFrame", dataframe=df)


KeyboardInterrupt: 

In [19]:
import pandas as pd

# Assuming trial_data is already defined and populated

# Flatten the trial data into a structured format
rows = []

for trial in trial_data:
    trial_id = trial["trial_id"]
    start_time = trial["start_time"]
    stop_time = trial["stop_time"]
    trial_type = trial["trial_type"]
    target_pos = trial["target_pos"]
    barrier_pos = trial["barrier_pos"]

    # Iterate over each timestamped entry in this trial
    for i, timestamp in enumerate(trial["hand_timestamps"]):
        row = {
            "trial_id": trial_id,
            "timestamp": timestamp,
            "hand_pos": trial["hand_pos"][i] if i < len(trial["hand_pos"]) else None,
            "hand_vel": trial["hand_vel"][i] if i < len(trial["hand_vel"]) else None,
            "eye_pos": trial["eye_pos"][i] if i < len(trial["eye_pos"]) else None,
            "cursor_pos": trial["cursor_pos"][i] if i < len(trial["cursor_pos"]) else None,
            "target_pos": target_pos,
            "barrier_pos": barrier_pos,
            "trial_type": trial_type,
            "start_time": start_time,
            "stop_time": stop_time,
        }

        # Add spike data for each of the 142 neurons/channels
        for neuron_idx in range(142):  # Assuming 142 spike channels
            spike_col = f"spike_{neuron_idx + 1}"
            row[spike_col] = trial.get("spikes", {}).get(neuron_idx, {}).get("data", [None])[i]

        rows.append(row)

# Convert the list of rows into a Pandas DataFrame
structured_data = pd.DataFrame(rows)

# Save to CSV for further analysis
output_path = "spike_timestamps.csv"
structured_data.to_csv(output_path, index=False)

# Display the DataFrame and output path to the user



KeyError: 'hand_timestamps'

In [20]:
import pandas as pd

# Flatten the trial data into a structured format
rows = []

for trial in trial_data:
    trial_id = trial["trial_id"]
    start_time = trial["start_time"]
    stop_time = trial["stop_time"]
    trial_type = trial["trial_type"]
    target_pos = trial["target_pos"]
    barrier_pos = trial["barrier_pos"]

    # Retrieve timestamps from the hand_pos.timestamps
    trial_timestamps = hand_pos.timestamps
    trial_hand_pos = trial["hand_pos"]
    trial_hand_vel = trial["hand_vel"]
    trial_eye_pos = trial["eye_pos"]
    trial_cursor_pos = trial["cursor_pos"]

    for i, timestamp in enumerate(trial_timestamps):
        # Ensure the timestamp falls within the trial's interval
        if start_time <= timestamp <= stop_time:
            row = {
                "trial_id": trial_id,
                "timestamp": timestamp,
                "hand_pos": trial_hand_pos[i] if i < len(trial_hand_pos) else None,
                "hand_vel": trial_hand_vel[i] if i < len(trial_hand_vel) else None,
                "eye_pos": trial_eye_pos[i] if i < len(trial_eye_pos) else None,
                "cursor_pos": trial_cursor_pos[i] if i < len(trial_cursor_pos) else None,
                "target_pos": target_pos,
                "barrier_pos": barrier_pos,
                "trial_type": trial_type,
                "start_time": start_time,
                "stop_time": stop_time,
            }

            # Add spike data for each of the 142 neurons/channels
            for neuron_idx in range(142):  # Assuming 142 spike channels
                if neuron_idx < len(trial["spikes"]):
                    spike_data = trial["spikes"][neuron_idx]
                    row[f"spike_{neuron_idx + 1}"] = spike_data[i] if i < len(spike_data) else None
                else:
                    row[f"spike_{neuron_idx + 1}"] = None

            rows.append(row)

# Convert the list of rows into a Pandas DataFrame
structured_data = pd.DataFrame(rows)

# Save to CSV for further analysis
output_path = "spike_timestamps.csv"
structured_data.to_csv(output_path, index=False)

# Display the DataFrame and output path to the user


KeyError: 'spikes'

In [21]:
import pandas as pd

# Flatten the trial data into a structured format
rows = []

for trial in trial_data:
    trial_id = trial["trial_id"]
    start_time = trial["start_time"]
    stop_time = trial["stop_time"]
    trial_type = trial["trial_type"]
    target_pos = trial["target_pos"]
    barrier_pos = trial["barrier_pos"]

    # Retrieve timestamps from the hand_pos.timestamps
    trial_timestamps = hand_pos.timestamps
    trial_hand_pos = trial["hand_pos"]
    trial_hand_vel = trial["hand_vel"]
    trial_eye_pos = trial["eye_pos"]
    trial_cursor_pos = trial["cursor_pos"]

    for i, timestamp in enumerate(trial_timestamps):
        # Ensure the timestamp falls within the trial's interval
        if start_time <= timestamp <= stop_time:
            row = {
                "trial_id": trial_id,
                "timestamp": timestamp,
                "hand_pos": trial_hand_pos[i] if i < len(trial_hand_pos) else None,
                "hand_vel": trial_hand_vel[i] if i < len(trial_hand_vel) else None,
                "eye_pos": trial_eye_pos[i] if i < len(trial_eye_pos) else None,
                "cursor_pos": trial_cursor_pos[i] if i < len(trial_cursor_pos) else None,
                "target_pos": target_pos,
                "barrier_pos": barrier_pos,
                "trial_type": trial_type,
                "start_time": start_time,
                "stop_time": stop_time,
            }

            # Add spike data for each unit (assuming spike_times is accessible)
            for neuron_idx, unit_spike_times in enumerate(units["spike_times"]):
                # Count spikes within a small window around the current timestamp
                spikes_at_time = len([t for t in unit_spike_times if start_time <= t <= stop_time])
                row[f"spike_{neuron_idx + 1}"] = spikes_at_time

            rows.append(row)

# Convert the list of rows into a Pandas DataFrame
structured_data = pd.DataFrame(rows)

# Save to CSV for further analysis
output_path = "/content/drive/MyDrive/MLNBD/ML_Neuro/DATA ANALYTICS/spikes.csv"
structured_data.to_csv(output_path, index=False)




KeyboardInterrupt: 